In [0]:
# carregando o dataset
import pandas as pd
url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/titanic_%20train.csv' 
titanic = pd.read_csv(url)

# mudado a ordem das colunas para que o y seja a ultima coluna 
titanic = titanic[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived']]

# definindo id do passageiro como índice 

In [0]:
# verificando o tipo dos dados
titanic.dtypes

# muito dados faltantes em várias linhas 
# dropna talvez deixe o modelo muito pobre em dados de treino
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Survived       891 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [0]:
# lidando com os dados faltantes na coluna escolhida
for column in titanic.columns:
    if titanic[column].dtype != 'object':
        titanic[column].fillna(titanic[column].mean(), inplace= True)
    else:
        titanic[column] = titanic[column].astype('str')
        titanic[column].fillna(titanic[column].mode(), inplace= True)
        

In [0]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null object
Survived       891 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [0]:
## preparando os dados para os modelos
#definindo as colunas a serem usadas no modelo
colunas = ['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked', 'Survived']

# colunas para teste
colunas2 = colunas.copy()
colunas2.remove('Survived')

# separando o dataset em variáveis independentes e dependentes
titanic_x = titanic.copy()[colunas].drop('Survived', axis= 1)
titanic_y = titanic.copy()[colunas]['Survived']

# codificando as variaveis categoricas
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for column in titanic_x.columns:
    if titanic_x[column].dtype == 'object':
        titanic_x[column] = le.fit_transform(titanic_x[column])

In [0]:
## ajuste e avaliaçào do modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score


# chamando o classificador 
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# perfomando o cross validation no dataset segundo do classificador
cv_score = cross_val_score(clf, titanic_x, titanic_y, cv= 10, scoring= 'accuracy')

#avaliando os resultados
print('Mean accuracy cv \n', cv_score.mean())


C:\Users\Avell\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Mean accuracy cv 
 0.8026937918510952


In [0]:
## treinamento do modelo
# ajustando o modelo
from sklearn.model_selection import train_test_split

# dividindo os dados em treino e teste para fazer a avaliação mais acurada de outras métricas
x_treino, x_teste, y_treino, y_teste = train_test_split(titanic_x, titanic_y, test_size= 0.20, random_state= 55)

# ajustando o modelo preditivo aos dados de treino
clf.fit(x_treino, y_treino)

# prevendo as saídas nos dados de teste
resultado = clf.predict(x_teste)

# avaliaçào dos resultados
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

print('Confusion matrix \n', confusion_matrix(resultado, y_teste), '\n',
     'Accuracy score \n', accuracy_score(resultado, y_teste), '\n', 
     'Recall score \n', recall_score(resultado, y_teste), '\n',
     'Precision score \n', precision_score(resultado, y_teste), '\n',
     'f1 score \n', f1_score(resultado, y_teste))



Confusion matrix 
 [[99 29]
 [11 40]] 
 Accuracy score 
 0.776536312849162 
 Recall score 
 0.7843137254901961 
 Precision score 
 0.5797101449275363 
 f1 score 
 0.6666666666666666


In [0]:
url_teste = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/titanic_test.csv'
teste = pd.read_csv(url_teste)

In [0]:
# teste para rodar o modelo
teste = teste[colunas2]
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [0]:
# lidando com os valores faltantes
for column in teste.columns:
    if teste[column].dtype != 'object':
        teste[column].fillna(teste[column].mean(), inplace= True)
    else:
        teste[column] = teste[column].astype('str')
        teste[column].fillna(teste[column].mode(), inplace= True)
        

In [0]:
# reaplicado a tranformação do label encoder
le = LabelEncoder()
for column in teste.columns:
    if teste[column].dtype == 'object':
        teste[column] = le.fit_transform(teste[column])

In [0]:
# resuldado da primeira previsão
resultado = clf.predict(teste)

In [0]:
"""
## montado o dataset resposta para ser enviado
# subido o template
df = pd.read_csv(url_teste)
envio = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': resultado})

# salvando um csv
nome_arquivo = 'titanic_submission_1.csv'
envio.to_csv(nome_arquivo, index= False)

#Fim
"""

"\n## montado o dataset resposta para ser enviado\n# subido o template\ndf = pd.read_csv(url_teste)\nenvio = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': resultado})\n\n# salvando um csv\nnome_arquivo = 'titanic_submission_1.csv'\nenvio.to_csv(nome_arquivo, index= False)\n\n#Fim\n"

In [0]:
## ajuste de um segundo modelo de privisão
